In [169]:
from class_item import Item
import copy

eps = 1

poligon1 = [[0,1,0], [1,1,1], [0,1,0]]
poligon2 = [[1,1], [1,0]]
poligon3 = [[1,1,1], [0,1,0]]

points_1 = [[0,3], [2,0], [3,1], [2,1], [1, 4]]
points_2 = [[0,3], [2,0], [3,1], [2,1], [2, 2]]
points_3 = [[0,3], [2,0], [2,1], [3,2], [1, 4]]

poligon_matrix = [[1,1,1,1], [1,1,1,1], [1,1,1,1], [1,1,1,1]]

poligons = [poligon1, poligon2, poligon3, poligon1, poligon2, poligon3, poligon1, poligon2, poligon3, poligon1, poligon2, poligon3]

items = [Item(i, poligons[i]) for i in range(len(poligons))]

for item in items:
    item.matrix = poligon_matrix

matrix = [[0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0]]
pallets = []

In [170]:
def print_matrix(matrix):
    for i in range(len(matrix)):
        print(matrix[i])
        print('\n')


print_matrix(matrix)


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]




In [171]:
def fit_item(pallet, item):
    for i in range(len(pallet[0])):
        for j in range(len(pallet)):
            exit = False
            # условия, что объект влезает
            if len(item.matrix) + i <= len(pallet) and len(item.matrix[0]) + j <= len(pallet[0]):
                # располагаем объект
                for p in range(len(item.matrix)):
                    for k in range(len(item.matrix[0])):
                        # решаем есть ли пересечение
                        if pallet[i+p][j+k] + item.matrix[p][k] == 2:
                            exit = True
                            break
                    if exit:
                        break
            else:
                exit = True
            # если пересечений нет и элемент влезает, то добавляем его
            if not exit:
                item.lb_x = i
                item.lb_y = j
                for p in range(len(item.matrix)):
                    for k in range(len(item.matrix[0])):
                        pallet[i+p][j+k] += item.matrix[p][k]
                break
        if not exit:
            break
    # print_matrix(pallet)
    # print('--------------')
    return pallet, exit

In [172]:
# fit_item(matrix, poligon2)

In [173]:
def fit_list(matrix, items):
    for item in items:
        matrix, exit = fit_item(matrix, item)
        # print_matrix(matrix)
        # print('--------------')
    return matrix

In [174]:
# fit_list(matrix, poligons)

In [175]:
def fit_pallets(matrix, pallets, items):
    pallets.append(copy.deepcopy(matrix))
    print_matrix(matrix)
    print('+++++++++++++++')
    for item in items:
        poligon = item.points
        i=0
        exit = True
        while exit and i<len(pallets):
            pallets[i], exit = fit_item(pallets[i], item)
            if exit and i==(len(pallets)-1):
                pallets.append(copy.deepcopy(matrix))
            if not exit:
                item.pallet_number = i+1
            i+=1
    return pallets

In [176]:
def find_lb_coordinates(item, eps):
    item.lb_x = item.lb_x * eps
    item.lb_y = item.lb_y * eps
    return item

In [177]:
fit_pallets(matrix, pallets, items)

find_lb_coordinates(items, eps)

print(items[3].pallet_number)

[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


+++++++++++++++
[1, 1, 1, 1, 0, 0, 0, 0]


[1, 1, 1, 1, 0, 0, 0, 0]


[1, 1, 1, 1, 0, 0, 0, 0]


[1, 1, 1, 1, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


--------------
[1, 1, 1, 1, 1, 1, 1, 1]


[1, 1, 1, 1, 1, 1, 1, 1]


[1, 1, 1, 1, 1, 1, 1, 1]


[1, 1, 1, 1, 1, 1, 1, 1]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


--------------
[1, 1, 1, 1, 1, 1, 1, 1]


[1, 1, 1, 1, 1, 1, 1, 1]


[1, 1, 1, 1, 1, 1, 1, 1]


[1, 1, 1, 1, 1, 1, 1, 1]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


--------------
[1, 1, 1, 1, 0, 0, 0, 0]


[1, 1, 1, 1, 0, 0, 0, 0]


[1, 1, 1, 1, 0, 0, 0, 0]


[1, 1, 1, 1, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0,

AttributeError: 'list' object has no attribute 'lb_x'